A survey of egraphs for compilers

Backwards search for egg is a good start



https://egraphs.org/meeting/2024-09-19-isaria 
https://dl.acm.org/doi/10.1145/3617232.3624873 Automatic Generation of Vectorizing Compilers for Customizable Digital Signal Processors

Latent Idiom Recognition for a Minimalist Functional Array Language using Equality Saturation https://arxiv.org/abs/2312.17682

https://egraphs.org/meeting/2024-04-18-sparse-tensor-diff
https://arxiv.org/pdf/2303.07030  ∇SD: Differentiable Programming for Sparse Tensors
